In [18]:
import pandas
import requests
from Bio import Entrez
from Bio.KEGG import REST
from Bio import SeqIO
import os

In [19]:
Entrez.email = "ppi_project@protonmail.me"
transcriptome = pandas.read_csv("Data/saccharomyces_cerevisiae.tsv", sep="\t")
proteome = transcriptome.loc[transcriptome["Gene Type"] == "protein-coding"]
proteome

,Accession,Begin,End,Chromosome,Orientation,Name,Symbol,Gene ID,Gene Type,Transcripts accession,Protein accession,Protein length,Locus tag
0,NC_001133.9,1807,2169,I,minus,seripauperin PAU8,PAU8,851229,protein-coding,NM_001180043.1,NP_009332.1,120.0,YAL068C
1,NC_001133.9,2480,2707,I,plus,uncharacterized protein,NaN,1466426,protein-coding,NM_001184582.1,NP_878038.1,75.0,YAL067W-A
2,NC_001133.9,7235,9016,I,minus,putative permease SEO1,SEO1,851230,protein-coding,NM_001178208.1,NP_009333.1,593.0,YAL067C
3,NC_001133.9,11565,11951,I,minus,uncharacterized protein,NaN,851232,protein-coding,NM_001179897.1,NP_009335.1,128.0,YAL065C
4,NC_001133.9,12046,12426,I,plus,uncharacterized protein,NaN,851233,protein-coding,NM_001180042.1,NP_009336.1,126.0,YAL064W-B
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6443,NC_001224.1,48901,50097,MT,plus,mitochondrial 37S ribosomal protein VAR1,VAR1,854586,protein-coding,NaN,NP_009320.1,398.0,Q0140
6445,NC_001224.1,61022,61729,MT,plus,intron-encoded endonuclease I-SceI,SCEI,854590,protein-coding,NaN,NP_009324.1,235.0,Q0160
6462,NC_001224.1,73758,74513,MT,plus,cytochrome c oxidase subunit 2,COX2,854622,protein-coding,NaN,NP_009326.1,251.0,Q0250
6463,NC_001224.1,74495,75984,MT,plus,maturase-like protein,NaN,854623,protein-coding,NaN,NP_009327.1,472.0,Q0255


In [20]:
for acc in proteome["Protein accession"]:
    with open(f"Data/Sequence/{acc}.txt", "w") as filout:
        filout.write(Entrez.efetch(db="protein", id=acc, rettype="fasta", retmode="text").read())

KeyboardInterrupt: 

In [ ]:
with open("Data/pathways.txt", "r") as fil:
    pathways = list(map(lambda x: x.split("\t"), fil.readlines()))

In [ ]:
for pt in pathways:
    with open(f"Data/Pathways/{pt[1][:-1]}.txt", "w") as filout:
        texte = REST.kegg_get(pt[0]).read()
        if "GENE" in texte:
            texte = texte[texte.find("GENE")+4:]
            if "COMPOUND" in texte:
                texte = texte[:texte.find("COMPOUND")-1].split("\n")
            else:
                texte = texte[:texte.find("REFERENCE")-1].split("\n")
            t = '\n'.join(list(map(lambda x: x.split()[0], texte)))
        filout.write(t)

In [30]:
with open("Data/kegg_uniprot.txt", "r") as fil:
    gids = list(map(lambda x: x.replace("\n", "").split("\t"), fil.readlines()))

In [34]:
rep = list(map(lambda x: x[:-4], os.listdir("Data/Gene Ontology")))


for gid in gids:
    if gid[1][4:] not in rep:
        texte_go = "GO id;GO term\n"
        texte_domaine = "Domain type;Domaine Description;Domain begin;Domain end\n"
        texte_structure = "Structure type;Structure begin;Structure end\n"

        reponse = requests.get(f"https://www.ebi.ac.uk/proteins/api/proteins/{gid[0][3:]}")
        reponse = reponse.json()

        for sous in reponse["dbReferences"]:
            if sous["type"] == "GO":
                try:
                    texte_go += f"{sous["id"]};{sous["properties"]["term"]}\n"
                except:
                    pass

        for sous in reponse["features"]:
            if sous["category"] == "DOMAINS_AND_SITES":
                try:
                    texte_domaine += f"{sous["type"]};{sous["description"]};{sous["begin"]};{sous["end"]}\n"
                except:
                    pass

            elif sous["category"] == "STRUCTURAL":
                try:
                    texte_structure += f"{sous["type"]};{sous["begin"]};{sous["end"]}\n"
                except:
                    pass
                
        with open(f"Data/Gene Ontology/{gid[1][4:]}.txt", "w") as fg, open(f"Data/Domain/{gid[1][4:]}.txt", "w") as fd, open(f"Data/Structure/{gid[1][4:]}.txt", "w") as fs:
            fg.write(texte_go)
            fd.write(texte_domaine)
            fs.write(texte_structure)

In [ ]:
#structure, goterm